# Analysis of datacube data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import plotPS as pps 
from figure import CustomFigure

Some information about the boxes and limiting frequencies:

In [ ]:
boxsize = 5120 #Mpc
ngrid = 256 #px
resolution = boxsize/ngrid #Mpc/px
k_nyquist = np.pi / resolution 
k_boxsize = 2*np.pi/boxsize

I will denote the power spectra outputed by gevolution with a superscript $gev$, by class with $cls$ and camb $cmb$. Thus, we have the following quantities:

* $P_\delta^{gev}(k)$ -> Matter power spectrum from gevolution
* $P_\delta^{cls}(k)$ -> Matter power spectrum from CLASS
* $P_\delta^{cmb}(k)$ -> Matter power spectrum from CAMB
* $P_\phi^{gev}(k)$ -> Potential power spectrum from gevolution
* $P_\phi(k)$ -> Potential power spectrum directly calculated from the datacube, using Pylians

We start off with plotting the matter power spectra generated by gevolution itself, $P_\delta^{gev}(k)$, for three different seeds and at three different redshifts, namely $z\in[0,1,10]$

In [ ]:
fig, ax = plt.subplots()
x = np.linspace(0,10,100)
line = Line2D(x, np.sin(x), label="test", color="red")

ax.add_line(line)
# ax.set_xlim(0,10)
# ax.set_ylim(-1,1)
leg1 = plt.legend(handles=[line], loc="upper right")
ax.add_artist(leg1)
ax.autoscale_view()


In [ ]:
class Test:
    def __init__(self):
        self.test1 = 1

    def __call__(self):
        if not hasattr(self, "test1"):
            self.test1 = 2
        if not hasattr(self, "test2"):
            self.test2 = 3
        print(self.test1, self.test2)

T = Test()
T()

In [ ]:
datapath = "/mn/stornext/d10/data/johanmkr/simulations/gevolution_first_runs/"

seeds = [0000, 1234, 1999]
pk_type = "delta" 
redshift = 0
ps_objects = [pps.PlotPowerSpectra(datapath +f"seed{seeds[i]:04d}/") for i in range(len(seeds))]

def compare_seeds(ps_objects, pk_type, redshift):
    temp_fig = CustomFigure()
    colors = ["r", "g", "b"]
    settings = {
            "xscale": "log",
            "yscale": "log",
            "xlabel": r"k $[h/Mpc]$",
            "ylabel": r"P(k) $[h/Mpc]^3$",
            "title": f"Power spectrum of '{pk_type} at redshift z={redshift:.1f}, for seeds {seeds}",
            "xlim": (k_boxsize, k_nyquist),
            "ylim": (1e0, 1e5)
        }
    temp_fig.set_settings(settings)
    lines = []
    for i in range(len(ps_objects)):
        grline, _ = ps_objects[i]._add_gr_newton(ax=temp_fig.ax, pk_type=pk_type, redshift=redshift, add_ratio=False, color=colors[i])
        lines.append(grline)
    
    cambline, classline = ps_objects[i]._add_camb_class(ax=temp_fig.ax, redshift=redshift)
    lines.append(cambline)
    lines.append(classline)
    temp_fig.ax.legend(lines, seeds+["CAMB", "CLASS"])


compare_seeds(ps_objects, "delta", redshift=0)
compare_seeds(ps_objects, "delta", redshift=1)
compare_seeds(ps_objects, "delta", redshift=10)


We can plot the same for the potential, but now 